# Задача: исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


**Описание данных**<br>
children — количество детей в семье<br>
days_employed — общий трудовой стаж в днях<br>
dob_years — возраст клиента в годах<br>
education — уровень образования клиента<br>
education_id — идентификатор уровня образования<br>
family_status — семейное положение<br>
family_status_id — идентификатор семейного положения<br>
gender — пол клиента<br>
income_type — тип занятости<br>
debt — имел ли задолженность по возврату кредитов<br>
total_income — ежемесячный доход<br>
purpose — цель получения кредита<br>

## 1. Создание датафрейма

In [1]:
import pandas as pd
df = pd.read_csv("./data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Предобработка данных

### 2.1 Обработка пропусков

In [2]:
def age_grouping(row):
    ages = row['dob_years']
    if ages <= 21:
        return "Юный возраст"
    elif ages > 21 and ages <= 35:
        return "Зрелый возраст I"
    elif ages > 35 and ages <= 57:
        return "Зрелый возраст II"
    else:
        return "Пожилой возраст"

In [3]:
df['age_group'] = df.apply(age_grouping, axis=1)

In [4]:
df['days_employed'] = df['days_employed'].fillna(df.groupby(['gender','age_group'])['days_employed'].transform('median'))

In [5]:
df['total_income'] = df['total_income'].fillna(df.groupby(['gender','age_group'])['total_income'].transform('median'))

**Вывод**<br>
Cкорее всего просто говорить что раз нет рабочего стажа значит и нет никаких заработков скорее всего не верно, тк все люди так или иначе имели работу за исключением пенсионеров, но им полагается пенсия так что доход должен быть видимо у всех групп где были пропуски, поэтому я решил что нужно заменить все пропущенные значения на медианы значений из соответсвущих групп, не стал такого делать только лишь для предпренимателя т.к их всего 2-е(двое) и если я возьму значения из столбцов второго предпринимателя то это будет скорее всего не верно т.к медиана скорее всего не будет отражать реальности. Также просто потеря данных не очень хорошо т.к мы теоретически теряем данные для будущей статистики, поэтому замена данных самая логичная стратегия

### 2.2 Замена типа данных

In [6]:
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
 12  age_group         21525 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


**Вывод**

Т.к мне не нужно переводить из str в float, а из float к int то нужно использовать метод astype()

### 2.3 Обработка дубликатов

In [8]:
df["education"].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [9]:
df["education"] = df["education"].str.lower()

In [10]:
df['family_status'] = df['family_status'].str.lower()

In [11]:
df["income_type"] = df['income_type'].str.lower()

In [12]:
df["purpose"] = df["purpose"].str.lower()

In [13]:
df["days_employed"] = df["days_employed"].abs()

In [14]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Зрелый возраст II
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Зрелый возраст II
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Зрелый возраст I
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Зрелый возраст I
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Зрелый возраст II
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,Зрелый возраст I
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,Зрелый возраст II
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,Зрелый возраст II
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,Зрелый возраст I
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,Зрелый возраст II


In [15]:
df["education"].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [16]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [17]:
df["duplica"] = df.duplicated()

In [18]:
df[df['duplica'] == True].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,duplica
2849,0,1716,41,среднее,1,женат / замужем,0,F,сотрудник,0,138323,покупка жилья для семьи,Зрелый возраст II,True
3290,0,354969,58,среднее,1,гражданский брак,1,F,пенсионер,0,122038,сыграть свадьбу,Пожилой возраст,True
4182,1,1285,34,высшее,0,гражданский брак,1,F,сотрудник,0,134654,свадьба,Зрелый возраст I,True
4851,0,354969,60,среднее,1,гражданский брак,1,F,пенсионер,0,122038,свадьба,Пожилой возраст,True
5557,0,354969,58,среднее,1,гражданский брак,1,F,пенсионер,0,122038,сыграть свадьбу,Пожилой возраст,True
6312,0,1135,30,среднее,1,женат / замужем,0,M,сотрудник,0,167989,строительство жилой недвижимости,Зрелый возраст I,True
7808,0,1716,57,среднее,1,гражданский брак,1,F,пенсионер,0,138323,на проведение свадьбы,Зрелый возраст II,True
7921,0,354969,64,высшее,0,гражданский брак,1,F,пенсионер,0,122038,на проведение свадьбы,Пожилой возраст,True
7938,0,354969,71,среднее,1,гражданский брак,1,F,пенсионер,0,122038,на проведение свадьбы,Пожилой возраст,True
8583,0,354969,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,122038,дополнительное образование,Пожилой возраст,True


In [19]:
df = df.drop_duplicates().reset_index(drop=True)

In [20]:
df.duplicated().sum()

0

**Вывод** 


Причина возникновения дупликатов не совсем ясна, но я предполгаю что просто одинаковые люди приходили брали кредит, на одни и те же цели. А сами дупликаты были найдены с помощью того что были все строчные столбцы приведены к одному регистру и заменены отрицательные значения столбцов на положительные как в столбце days_employed, где явно была допущена ошибка при заполнение.

### 2.4 Лемматизация

In [21]:
from pymystem3 import Mystem
m = Mystem()

In [22]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     775
сыграть свадьбу                           773
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              610
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [23]:
lemmas = []

In [24]:
for i in range(1,df.shape[0]):
    all = m.lemmatize(df.loc[i,'purpose'])
    for b in all:
        lemmas.append(b)

In [25]:
from collections import Counter
print(Counter(lemmas))

Counter({' ': 33669, '\n': 21519, 'недвижимость': 6367, 'покупка': 5911, 'жилье': 4470, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2345, 'свой': 2235, 'на': 2231, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 775, 'сыграть': 773, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 610, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


**Вывод**

Лемматизация была проведена по всем словам из колонки 'purpose' путем слияния всех лемм в один спискок и дальнейшим подсчётом слов, таким образом можно увидеть какие основные позиции можно выделить в дальнейшем для категоризации данных такие как :недвижимость/жилье, образование, автомобиль, Свадьба

### 2.5 Категоризация данных

In [26]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
def _group_(row):
    
    purp = row['purpose']
    if ('недвижимость' in m.lemmatize(purp)) or ("жилье" in m.lemmatize(purp)):
        return "Операции Жильё/Недвижимость"
    elif "автомобиль" in m.lemmatize(purp):
        return "Операции с автомобилями"
    elif "образование" in m.lemmatize(purp):
        return "Образованием"
    elif "свадьба" in m.lemmatize(purp):
        return "Свадьба"
    else:
        return "other's"

In [27]:
df['group_purpose'] = df.apply(_group_,axis=1)

In [28]:
df["group_purpose"].value_counts()

Операции Жильё/Недвижимость    10838
Операции с автомобилями         4315
Образованием                    4022
Свадьба                         2345
Name: group_purpose, dtype: int64

**Вывод**

Можно резделить все выбранные цели на 4 категории с которыми в дальнейшем можно производить проверку гипотез

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
df.groupby("children")["debt"].sum()/df.groupby('children')['debt'].count() * 100

children
-1      2.127660
 0      7.515554
 1      9.215442
 2      9.440389
 3      8.181818
 4      9.756098
 5      0.000000
 20    10.526316
Name: debt, dtype: float64

**Вывод**<br>

Была проведен перевод значений в проценты путем деления на количество людей в каждой группе и умножения на 100 => В среднем те кто имеют детей склонны чаще не возвращать деньги в срок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
df['family_status'].value_counts()

женат / замужем          12378
гражданский брак          4174
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [31]:
sum = df.groupby("family_status")['debt'].sum()

In [32]:
sum

family_status
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
не женат / не замужем    274
Name: debt, dtype: int64

In [33]:
value_counts = df.groupby('family_status').count()['debt']

In [34]:
value_counts

family_status
в разводе                 1195
вдовец / вдова             960
гражданский брак          4174
женат / замужем          12378
не женат / не замужем     2813
Name: debt, dtype: int64

In [35]:
sum/value_counts

family_status
в разводе                0.071130
вдовец / вдова           0.065625
гражданский брак         0.092956
женат / замужем          0.075214
не женат / не замужем    0.097405
Name: debt, dtype: float64

**Вывод**

Можно заметить что когда человек не женат у него больше вероятность не заплатить во время, вообще видно мне кажется что в зависимости от того женат был человек или нет очень повышается вероятность т.е самая низкая у вдовцов т.к они были когда то женаты, потом в разводе т.к человек был когда то женат, затем идут женатые, затем идут не женатые или находящиеся в гражданском браке => Женатому человеку можно давать в кредит он не 'подведет', не женатому с опасением)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
sorted_df = df.sort_values(by="total_income")["total_income"].reset_index(drop=True)

In [37]:
len_total_income = len(df['total_income'])
first_value = sorted_df[int(len_total_income * 0.1) - 1]
second_value = sorted_df[int(len_total_income * 0.25) - 1]
third_value = sorted_df[int(len_total_income * 0.75) - 1]
print(first_value, " " , second_value, " ", third_value)

78796   107780   195549


In [38]:
def sort_income(row):
    income = row['total_income'] 
    if income < 78785:
        return 'низкий'
    elif income >= 78785 and income < 107775:
        return "ниже_среднего"
    elif income >= 107775 and income < 195549:
        return "средний"
    else:
        return "высокий"

In [39]:
df['sort_income'] = df.apply(sort_income, axis=1)

In [40]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,duplica,group_purpose,sort_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Зрелый возраст II,False,Операции Жильё/Недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Зрелый возраст II,False,Операции с автомобилями,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Зрелый возраст I,False,Операции Жильё/Недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Зрелый возраст I,False,Образованием,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Зрелый возраст II,False,Свадьба,средний


In [41]:
df.groupby("sort_income")["debt"].sum()/df.groupby('sort_income')['debt'].count() * 100

sort_income
высокий          7.173388
ниже_среднего    8.333333
низкий           7.348837
средний          8.623734
Name: debt, dtype: float64

**Вывод**<br>
Сначала я поделил заработок с на 4 категории: низкий, ниже среднего, средний, высокий. В интервал **низкие** зарплаты вхоидил люди с заработком из 10% диапазона отсортированных величин заработка.
В категорию **ниже среднего** от 10 до 25%, **средний**  от 25 до 75 %, **высокий** от 75% и выше. Таким образом с помощью функции для строки получилось отфилотровать все значения  .

Самыми **безответственными** оказались люди чей заработок входит в 'средний' диапазон, богатые же люди с 'высоким' заработком оказались самыми **ответственными** всери 4 групп

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
sum = df.groupby("group_purpose")['debt'].sum()

In [43]:
sum

group_purpose
Образованием                   370
Операции Жильё/Недвижимость    782
Операции с автомобилями        403
Свадьба                        186
Name: debt, dtype: int64

In [44]:
group_value = df.groupby("group_purpose").count()['children']

In [45]:
group_value

group_purpose
Образованием                    4022
Операции Жильё/Недвижимость    10838
Операции с автомобилями         4315
Свадьба                         2345
Name: children, dtype: int64

In [46]:
sum/group_value

group_purpose
Образованием                   0.091994
Операции Жильё/Недвижимость    0.072154
Операции с автомобилями        0.093395
Свадьба                        0.079318
dtype: float64

**Вывод**

В среднем те кто берет на образование и автомобили чаще не возвращают деньги в срок нежели остальные две группы

## Шаг 4. Общий вывод

Как и все реальные данные, и предоствленные здесь не исключение, все имеют дупликаты пропуски, и все они были убраны с помощью специальных методов таких как fillna, str.lower, drop_duplicates. 
Также были найдены отрацательные значения в столбце days_employed, что говорит нам что эти данные скорее возникли в следствие ошибки человека. Также я заметил что все цели кредита можно разбить на 4 категории а именно: 

-Образование

-Операции Жильё/Недвижимостью

-Операции с автомобилями

-Свадьба

После предварительной очистки данных были проверены 4 гипотезы, вследвтвие которых делается вывод о том что: 

1.видно, что чем больше детей тем меньше вероятность что деньги будут возвращены в срок (В среднем те кто имеют детей склонны чаще не возвращать деньги в срок)

2.женатые когда либо люди менее склонны к задежке оплаты в срок

3.Самыми безответственными оказались люди чей заработок входит в 'средний' диапазон чей уровень невозврата составляет **8.624535%**, богатые же люди с 'высоким' заработком оказались самыми ответственными среди 4 групп и их уровень невозврата составляет **7.173388%**

4.В среднем те кто берет на образование и автомобили чаще не возвращают деньги в срок нежели остальные две группы